### Caveat with Lazy Iterators

We have to be careful when working with context managers and lazy iterators.

Consider this example where we want to create a generator from a file:

In [5]:
import csv

def read_data():
    with open('nyc_parking_tickets_extract.csv') as f:
        return csv.reader(f, delimiter=',', quotechar='"')

In [6]:
for row in read_data():
    print(row)

ValueError: I/O operation on closed file.

As you can see, `read_data` returns a lazy iterator (`csv.reader`), but by the time we iterate over it, the `with` context that opened the file was exited, and the file was closed!

We have two possible solutions here:

The first one is not very desirable since it involves reading the entire file into memory by iterating the file and putting it into a list before we exit the `with` block:

In [7]:
def read_data():
    with open('nyc_parking_tickets_extract.csv') as f:
        return list(csv.reader(f, delimiter=',', quotechar='"'))

for row in read_data():
    print(row)

['Summons Number', 'Plate ID', 'Registration State', 'Plate Type', 'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make', 'Violation Description']
['4006478550', 'VAD7274', 'VA', 'PAS', '10/5/2016', '5', '4D', 'BMW', 'BUS LANE VIOLATION']
['4006462396', '22834JK', 'NY', 'COM', '9/30/2016', '5', 'VAN', 'CHEVR', 'BUS LANE VIOLATION']
['4007117810', '21791MG', 'NY', 'COM', '4/10/2017', '5', 'VAN', 'DODGE', 'BUS LANE VIOLATION']
['4006265037', 'FZX9232', 'NY', 'PAS', '8/23/2016', '5', 'SUBN', 'FORD', 'BUS LANE VIOLATION']
['4006535600', 'N203399C', 'NY', 'OMT', '10/19/2016', '5', 'SUBN', 'FORD', 'BUS LANE VIOLATION']
['4007156700', '92163MG', 'NY', 'COM', '4/13/2017', '5', 'VAN', 'FRUEH', 'BUS LANE VIOLATION']
['4006687989', 'MIQ600', 'SC', 'PAS', '11/21/2016', '5', 'VN', 'HONDA', 'BUS LANE VIOLATION']
['4006943052', '2AE3984', 'MD', 'PAS', '2/1/2017', '5', 'SW', 'LINCO', 'BUS LANE VIOLATION']
['4007306795', 'HLG4926', 'NY', 'PAS', '5/30/2017', '5', 'SUBN', 'TOYOT', 'BUS LANE

The second method, the one we have used quite a bit, involves yielding each row from the csv reader:

In [8]:
def read_data():
    with open('nyc_parking_tickets_extract.csv') as f:
        yield from csv.reader(f, delimiter=',', quotechar='"')

for row in read_data():
    print(row)

['Summons Number', 'Plate ID', 'Registration State', 'Plate Type', 'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make', 'Violation Description']
['4006478550', 'VAD7274', 'VA', 'PAS', '10/5/2016', '5', '4D', 'BMW', 'BUS LANE VIOLATION']
['4006462396', '22834JK', 'NY', 'COM', '9/30/2016', '5', 'VAN', 'CHEVR', 'BUS LANE VIOLATION']
['4007117810', '21791MG', 'NY', 'COM', '4/10/2017', '5', 'VAN', 'DODGE', 'BUS LANE VIOLATION']
['4006265037', 'FZX9232', 'NY', 'PAS', '8/23/2016', '5', 'SUBN', 'FORD', 'BUS LANE VIOLATION']
['4006535600', 'N203399C', 'NY', 'OMT', '10/19/2016', '5', 'SUBN', 'FORD', 'BUS LANE VIOLATION']
['4007156700', '92163MG', 'NY', 'COM', '4/13/2017', '5', 'VAN', 'FRUEH', 'BUS LANE VIOLATION']
['4006687989', 'MIQ600', 'SC', 'PAS', '11/21/2016', '5', 'VN', 'HONDA', 'BUS LANE VIOLATION']
['4006943052', '2AE3984', 'MD', 'PAS', '2/1/2017', '5', 'SW', 'LINCO', 'BUS LANE VIOLATION']
['4007306795', 'HLG4926', 'NY', 'PAS', '5/30/2017', '5', 'SUBN', 'TOYOT', 'BUS LANE